In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [39]:
bacia_mangue = pd.read_csv("data/email/rj-rioaguas.saneamento_drenagem.nivel_reservatorio.csv")
bacia_mangue.head()

,primary_key,id_reservatorio,nome_reservatorio,data_particao,horario,altura_agua
0,2_2021-11-30_23:59:51+00,2,Niteroi,2021-11-30,23:59:51,2.98
1,2_2021-11-30_21:59:52+00,2,Niteroi,2021-11-30,21:59:52,2.97
2,2_2021-11-30_19:59:56+00,2,Niteroi,2021-11-30,19:59:56,2.95
3,2_2021-11-30_17:59:39+00,2,Niteroi,2021-11-30,17:59:39,2.93
4,2_2021-11-30_15:59:39+00,2,Niteroi,2021-11-30,15:59:39,2.91


In [41]:
def remover_outliers(df):
    df_filtrado = df[df['altura_agua'] <= 23]
    return df_filtrado

bacia_mangue_filtrado = remover_outliers(bacia_mangue)

In [42]:
bacia_mangue_not_null = bacia_mangue_filtrado.dropna(how="any")

In [43]:
bacia_mangue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37836 entries, 0 to 37835
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   primary_key        37836 non-null  object 
 1   id_reservatorio    37836 non-null  int64  
 2   nome_reservatorio  37836 non-null  object 
 3   data_particao      37836 non-null  object 
 4   horario            37836 non-null  object 
 5   altura_agua        36440 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.7+ MB


In [44]:
bacia_mangue_not_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36293 entries, 0 to 37835
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   primary_key        36293 non-null  object 
 1   id_reservatorio    36293 non-null  int64  
 2   nome_reservatorio  36293 non-null  object 
 3   data_particao      36293 non-null  object 
 4   horario            36293 non-null  object 
 5   altura_agua        36293 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.9+ MB


In [45]:
print(bacia_mangue_not_null.columns)

Index(['primary_key', 'id_reservatorio', 'nome_reservatorio', 'data_particao',
       'horario', 'altura_agua'],
      dtype='object')


In [46]:
altura_maxima = {
    "Bandeira": 18,
    "Varnhagen": 21.5,
    "Niteroi": 22.25
}

def calcular_altura_ocupada(row):
    maxima = altura_maxima[row["nome_reservatorio"]]
    atual = row["altura_agua"]
    ocupada = (atual / maxima) * 100
    return ocupada

In [47]:
bacia_mangue_not_null.head(10)

,primary_key,id_reservatorio,nome_reservatorio,data_particao,horario,altura_agua
0,2_2021-11-30_23:59:51+00,2,Niteroi,2021-11-30,23:59:51,2.98
1,2_2021-11-30_21:59:52+00,2,Niteroi,2021-11-30,21:59:52,2.97
2,2_2021-11-30_19:59:56+00,2,Niteroi,2021-11-30,19:59:56,2.95
3,2_2021-11-30_17:59:39+00,2,Niteroi,2021-11-30,17:59:39,2.93
4,2_2021-11-30_15:59:39+00,2,Niteroi,2021-11-30,15:59:39,2.91
5,2_2021-11-30_13:59:50+00,2,Niteroi,2021-11-30,13:59:50,2.89
6,2_2021-11-30_11:59:29+00,2,Niteroi,2021-11-30,11:59:29,2.88
7,2_2021-11-30_09:59:31+00,2,Niteroi,2021-11-30,09:59:31,2.86
8,2_2021-11-30_07:59:35+00,2,Niteroi,2021-11-30,07:59:35,2.85
9,2_2021-11-30_05:59:30+00,2,Niteroi,2021-11-30,05:59:30,2.84


In [48]:
bacia_mangue_not_null["altura_ocupada"] = bacia_mangue_not_null.apply(calcular_altura_ocupada, axis=1)

df_agrupado = bacia_mangue_not_null.groupby(["data_particao", "nome_reservatorio"]).agg({
    "altura_ocupada": "mean"
}).reset_index()

df_agrupado = df_agrupado.rename(columns={'altura_ocupada': 'altura_ocupada_media'})

In [49]:
df_agrupado

,data_particao,nome_reservatorio,altura_ocupada_media
0,2021-01-04,Bandeira,9.352778
1,2021-01-04,Niteroi,4.116105
2,2021-01-04,Varnhagen,5.779845
3,2021-01-05,Bandeira,4.992130
4,2021-01-05,Niteroi,2.348315
...,...,...,...
3503,2024-03-21,Niteroi,0.359551
3504,2024-03-21,Varnhagen,0.346977
3505,2024-03-22,Bandeira,4.414815
3506,2024-03-22,Niteroi,0.404494


In [52]:
import plotly.express as px

df_agrupado["data_particao"] = pd.to_datetime(df_agrupado["data_particao"])

fig = px.line(df_agrupado, x="data_particao", y="altura_ocupada_media", color="nome_reservatorio", 
              labels={"data_particao": "Data", "altura_ocupada_media": "Altura Ocupada Média (%)", "nome_reservatorio": "Reservatório"},
              title="Altura Ocupada Média por Data para Cada Reservatório")

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.show()
